In [ ]:
# Add metric USA total exposure

In [1]:
import requests
import sqlalchemy
import pandas as pd
import logging
from concurrent.futures import ThreadPoolExecutor
import urllib3

# Suppress SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Database connection
connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)
engine = sqlalchemy.create_engine(connection_string)

# YCP API headers (exact match to your provided code)
headers_YCP = {
    "X-YCHARTSAUTHORIZATION": "yIIphqbsQysnTvWWxfW33w",
    "X-YCHARTSEXCELSESSION": "b645cd897b2446bfa3796acfa3a879db",
    "X-YCHARTSEXCELVERSION": "4.4",
    "X-YCHARTSOPERATINGSYSTEM": "Microsoft Windows NT 10.0.26100.0",
    "Content-Type": "application/x-www-form-urlencoded",
    "Host": "api.ycharts.com",
    "Connection": "Keep-Alive"
}

yc_base_url = "https://api.ycharts.com/v3/"
ycp_api_url = f"{yc_base_url}excel/points"
metric = "united_states_total_exposure"
db_column = "USA_Exposure"

# Fetch function for YCP metric
def fetch_ycp_metric(symbol, fund_type_id):
    symbol_prefix = f"M:{symbol}" if fund_type_id == 3 else symbol
    data = f"points={symbol_prefix},{metric}"
    logging.debug(f"YCP payload for {symbol}: {data}")
    
    try:
        response = requests.post(ycp_api_url, headers=headers_YCP, data=data, verify=False)
        if response.status_code != 200:
            logging.error(f"YCP HTTP error for {symbol}: {response.status_code}")
            return symbol, None
        
        data = response.json()
        response_key = symbol_prefix
        if "response" in data and response_key in data["response"]:
            results = data["response"][response_key].get("results", {})
            if metric in results and "" in results[metric] and "results" in results[metric][""]:
                data_list = results[metric][""]["results"]
                if isinstance(data_list, list) and len(data_list) > 1:
                    value = float(data_list[1]) if data_list[1] is not None else None
                    logging.debug(f"Extracted {metric} for {symbol}: {value}")
                    return symbol, value
        logging.warning(f"No valid data for {metric} in {symbol}")
        return symbol, None
    
    except requests.RequestException as e:
        logging.error(f"YCP request failed for {symbol}: {str(e)}")
        return symbol, None

# Database insertion function
def insert_to_database(df):
    try:
        with engine.begin() as conn:
            updates = [{"symbol": row["SymbolCUSIP"], "value": row[db_column]} for _, row in df.iterrows()]
            conn.execute(
                sqlalchemy.text(f"""
                    UPDATE Funds_to_Screen 
                    SET {db_column} = :value
                    WHERE SymbolCUSIP = :symbol
                """),
                updates
            )
        logging.info(f"Inserted {len(df)} rows for {metric}")
        return len(df), 0
    except sqlalchemy.exc.SQLAlchemyError as e:
        logging.error(f"Database error: {str(e)}")
        return 0, len(df)

# Main execution
def update_usa_exposure():
    # Fetch funds needing update
    query = f"SELECT SymbolCUSIP, Fund_Type_ID FROM Funds_to_Screen WHERE {db_column} IS NULL"
    funds = pd.read_sql(query, engine)
    logging.info(f"Found {len(funds)} funds needing {metric} update")
    
    if len(funds) == 0:
        print("No funds need USA Exposure updates.")
        return
    
    # Fetch data using ThreadPoolExecutor
    data_list = []
    with ThreadPoolExecutor(max_workers=60) as executor:
        future_to_symbol = {
            executor.submit(fetch_ycp_metric, row["SymbolCUSIP"], row["Fund_Type_ID"]): row["SymbolCUSIP"]
            for _, row in funds.iterrows()
        }
        for future in future_to_symbol:
            symbol, value = future.result()
            if value is not None:
                data_list.append((symbol, value))
    
    if not data_list:
        logging.info("No data retrieved from YCharts.")
        return
    
    # Insert into database
    df = pd.DataFrame(data_list, columns=["SymbolCUSIP", db_column])
    successes, failures = insert_to_database(df)
    
    # Summary
    print(f"Summary:\n  Funds needing update: {len(funds)}\n  Data retrieved: {len(df)}\n  Successful inserts: {successes}\n  Failed inserts: {failures}")

if __name__ == "__main__":
    update_usa_exposure()

2025-04-07 18:39:51,975 - INFO - Found 5586 funds needing united_states_total_exposure update
2025-04-07 18:39:55,708 - WARNING - No valid data for united_states_total_exposure in AMPD
2025-04-07 18:39:57,005 - WARNING - No valid data for united_states_total_exposure in VMOT
2025-04-07 18:40:03,081 - WARNING - No valid data for united_states_total_exposure in PP
2025-04-07 18:40:03,433 - WARNING - No valid data for united_states_total_exposure in VCAR
2025-04-07 18:40:03,724 - WARNING - No valid data for united_states_total_exposure in SQY
2025-04-07 18:40:32,098 - WARNING - No valid data for united_states_total_exposure in OFIYX
2025-04-07 18:40:32,833 - WARNING - No valid data for united_states_total_exposure in JFGIX
2025-04-07 18:40:46,388 - WARNING - No valid data for united_states_total_exposure in TWIO
2025-04-07 18:40:51,464 - WARNING - No valid data for united_states_total_exposure in DFNV
2025-04-07 18:40:53,625 - WARNING - No valid data for united_states_total_exposure in PR

Summary:
  Funds needing update: 5586
  Data retrieved: 5527
  Successful inserts: 5527
  Failed inserts: 0
